# Hermes — Run a basic Hartree-Fock energy calculation

See [the tutorial](/Tutorials/hermes_hartree_fock_energy_calculation.ipynb).

In [1]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

# Define our project information
EXPERIMENT = "rush-py hermes energy demo"
SYSTEM = "1B39"
LIGAND = "ATP"
TAGS = ["qdx", EXPERIMENT, SYSTEM, LIGAND]

# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(batch_tags=TAGS)

# RUSH_TOKEN and RUSH_URL are discovered automatically if present in the env.
# Alternatively, they can be set explicitly:
# TOKEN = os.getenv("RUSH_TOKEN")
# URL = os.getenv("RUSH_URL")
# client = await rush.build_provider_with_functions(access_token=TOKEN, url=URL, batch_tags=TAGS)

# Fetch datafiles
PROTEIN_PDB_PATH = client.workspace / f"{SYSTEM}_P.pdb"

# Preparation
complex = list(pdb_fetch.fetch_structure(SYSTEM))
protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))
# write our files to the locations defined in the config block
with open(PROTEIN_PDB_PATH, "w") as f:
    for l in protein:
        f.write(str(l))

(prepared_protein_qdxf, prepared_protein_pdb) = await client.prepare_protein(
    PROTEIN_PDB_PATH
)

print(f"{datetime.now().time()} | Running protein preparation!")

try:
    await prepared_protein_pdb.download(filename="01_prepared_protein.pdb")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file, you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

# Visualize the prepared protein to spot check any incorrectly transformed residues
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_protein.pdb", "r") as f:
    view.addModel(f.read(), "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})
    view.zoomTo()
    view.show()

# There may be multiple conformers, so select the first one
(first_conformer,) = await client.pick_conformer(
    prepared_protein_qdxf, 0
)

(fragmented_protein,) = await client.fragment_aa(
    first_conformer, 1, "All"
)

# Arguments
HERMES_RESOURCES = {
        "gpus": 1,
        "storage": 100,
        "storage_units": "MB",
        "walltime": 60,
}

fragmented_protein_out = await fragmented_protein.get()

# Quantum energy calculation
(hermes_energy,) = await client.hermes_energy(
    fragmented_protein,
    {},
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RestrictedRIMP2",
    },  # configuration for a fast converging, low accuracy run
    {
        "guess": {},
        "scf": {
            "max_iter": 50,
            "max_diis_history_length": 12,
            "convergence_metric": "DIIS",
            "dynamic_screening_threshold_exp": 10,
            "ndiis": 8,
            "niter": 40,
            "scf_conv": 0.000001,
        },
          "frag": {
        "cutoffs": {"dimer": 22},
        "cutoff_type": "Centroid",
        "level": "Dimer",
        "reference_fragment": len(fragmented_protein_out["topology"]["fragments"])
        - 1,
        },
    },
    resources=HERMES_RESOURCES,
)

energy = await hermes_energy.get()
# Let's look at the expanded Hartree Fock energy
energy['expanded_hf_energy']

# Log output
logs_output = client.logs(hermes_energy.source, kind="stdout", print_logs=False)

async for log_page in client.logs(
        hermes_energy.source, kind="stdout", print_logs=False
    ):
     for line in log_page[0:15]:
         print(line)
     break;

# If we wanted to view all the logs, we could do this:
    # async for log_page in client.logs(
    #     hermes_energy.source, kind="stdout", print_logs=False
    # ):
    #     for log in log_page:
    #         print(log)